In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget
plt.ion()
sns.set(rc={'figure.figsize': (11, 4)})
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# Load data
df = pd.read_csv("../data/imputed/data_imp1.csv", index_col=0)

In [4]:
# Drop last missing values
df = df.dropna()

In [5]:
# Group data by sitename
sites = df['sitename'].unique()
sites_df = [df[df['sitename'] == site] for site in sites]

for i in range(len(sites_df)):
    sites_df[i]['date'] = pd.to_datetime(sites_df[i]['date'], format="%Y-%m-%d")
    sites_df[i] = sites_df[i].set_index("date")


In [52]:
train_site = 3
predict_site = 1
model= sm.tsa.SARIMAX(dates=sites_df[train_site].index, freq='D', exog=sites_df[train_site][["TA_F", "SW_IN_F", "LW_IN_F", "VPD_F", "PA_F", "P_F", "WS_F", "USTAR", "CO2_F_MDS"]], endog=sites_df[train_site][["GPP_NT_VUT_REF"]], order=(2, 0, 0))

In [53]:
model = model.fit()

In [54]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:         GPP_NT_VUT_REF   No. Observations:                 2557
Model:               SARIMAX(2, 0, 0)   Log Likelihood               -5390.377
Date:                Sat, 27 Mar 2021   AIC                          10804.755
Time:                        17:30:55   BIC                          10874.914
Sample:                    01-01-2007   HQIC                         10830.197
                         - 12-31-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
TA_F           0.3395      0.071      4.757      0.000       0.200       0.479
SW_IN_F        0.0242      0.001     16.887      0.000       0.021       0.027
LW_IN_F       -0.0108      0.006     -1.720      0.085      -0.023       0.002
VPD_F         -0.3814      0.025    -15.011      0.000      -0.431      -0.332
PA_F           0.0024      0.017      0.139      0.889      -0.031       0.035
P_F           -0.0132      0.004     -3.037      0.002      -0.022      -0.005
WS_F          -0.5682      0.098     -5.787      0.000      -0.761      -0.376
USTAR          4.6222      0.304     15.198      0.000       4.026       5.218
CO2_F_MDS     -0.0026      0.002     -1.262      0.207      -0.007       0.001
ar.L1          0.4290      0.014     30.337      0.000       0.401       0.457
ar.L2          0.3492      0.014     24.753      0.000       0.322       0.377
sigma2         3.8872      0.078     49.838      0.000       3.734       4.040
===================================================================================
Ljung-Box (Q):                      297.92   Jarque-Bera (JB):              1371.20
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             0.07
Prob(H) (two-sided):                  0.00   Kurtosis:                         6.58
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [56]:
preds = model.predict(dates=sites_df[predict_site].index, freq='D', exog=sites_df[predict_site][["TA_F", "SW_IN_F", "LW_IN_F", "VPD_F", "PA_F", "P_F", "WS_F", "USTAR", "CO2_F_MDS"]], endog=sites_df[predict_site][["GPP_NT_VUT_REF"]])

index = sites_df[predict_site].index
preds = preds.values
truth = np.squeeze(sites_df[predict_site][["GPP_NT_VUT_REF"]].values)

# %matplotlib widget
# plt.ion()
# plt.scatter(index, preds, label="Predictions", s=1)
# plt.scatter(index, truth, label="Ground truth", s=1)
# plt.legend()
# plt.show()

In [50]:
residuals = (preds - truth) ** 2

%matplotlib widget
plt.ion()
plt.scatter(index, residuals, label="Residuals", s=1)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
np.sum(residuals) / len(index)

3.9667760653202784

In [57]:
len(preds)

2557

In [58]:
len(truth)

1096

In [59]:
len(sites_df[train_site].index)

2557

In [61]:
len(index)

1096